In [66]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_params # вывод результатов тестирования
from statsmodels.iolib.summary2 import summary_col # вывод результатов тестирования
from statsmodels.stats.diagnostic import linear_reset
from statsmodels.stats.outliers_influence import reset_ramsey
# import seaborn as sns
from scipy import stats
import re
# import scipy.stats as stats

In [67]:
source_file = 'sleep75.csv'
url = 'https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/'
df = pd.read_csv(url + source_file)
df

,age,black,case,clerical,construc,educ,earns74,gdhlth,inlf,leis1,...,spwrk75,totwrk,union,worknrm,workscnd,exper,yngkid,yrsmarr,hrwage,agesq
0,32,0,1,0.000000,0.000000,12,0,0,1,3529,...,0,3438,0,3438,0,14,0,13,7.070004,1024
1,31,0,2,0.000000,0.000000,14,9500,1,1,2140,...,0,5020,0,5020,0,11,0,0,1.429999,961
2,44,0,3,0.000000,0.000000,17,42500,1,1,4595,...,1,2815,0,2815,0,21,0,0,20.529997,1936
3,30,0,4,0.000000,0.000000,12,42500,1,1,3211,...,1,3786,0,3786,0,12,0,12,9.619998,900
4,64,0,5,0.000000,0.000000,14,2500,1,1,4052,...,1,2580,0,2580,0,44,0,33,2.750000,4096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,45,0,702,0.182331,0.030075,12,5500,1,0,5069,...,1,2026,0,2026,0,27,0,18,NaN,2025
702,34,0,703,0.182331,0.030075,10,2500,0,0,5885,...,0,675,1,465,210,18,0,4,NaN,1156
703,37,0,704,0.182331,0.030075,12,3500,1,0,4719,...,1,1851,0,1851,0,19,0,17,NaN,1369
704,54,0,705,0.182331,0.030075,17,32500,1,0,5149,...,1,1961,1,1481,480,31,0,22,NaN,2916


### 2 Сравнение моделей
#### (Вариант 2)

#### 2.1 sleep equations

Для набора данных `sleep75` рассмотрим несколько моделей регрессии.



In [68]:
mod = []

In [69]:
formula = 'sleep ~ totwrk + age + male + smsa + south'
mod.append(smf.ols(formula = formula, data = df).fit())
formula2 = formula + ' + I(totwrk**2)'
mod.append(smf.ols(formula = formula2, data = df).fit())
formula2 = formula + ' + I(age**2)'
mod.append(smf.ols(formula = formula2, data = df).fit())
formula2 = formula + ' + marr + union'
mod.append(smf.ols(formula = formula2, data = df).fit())
formula2 = formula + ' + I(totwrk**2) + I(age**2)'
mod.append(smf.ols(formula = formula2, data = df).fit())
formula2 = formula + ' + I(totwrk**2) + I(age**2) + marr + union'
mod.append(smf.ols(formula = formula2, data = df).fit())


In [70]:

print(
    summary_col(
        mod,
        float_format='%.3f',
        stars=True,
        model_names=['1', '2', '3', '4', '5', '6']
        # info_dict = info_dict
    )
)


                    1           2           3           4           5           6     
--------------------------------------------------------------------------------------
I(age ** 2)                            0.108                   0.119       0.130      
                                       (0.134)                 (0.133)     (0.134)    
I(totwrk ** 2)             -0.000*                             -0.000*     -0.000*    
                           (0.000)                             (0.000)     (0.000)    
Intercept      3470.460*** 3414.144*** 3638.626*** 3441.297*** 3597.061*** 3578.293***
               (69.377)    (76.730)    (218.595)   (78.942)    (219.572)   (220.730)  
R-squared      0.131       0.134       0.131       0.131       0.135       0.136      
R-squared Adj. 0.124       0.127       0.124       0.123       0.126       0.125      
age            2.831**     2.839**     -6.165      2.841**     -7.016      -7.977     
               (1.385)     (1.383)     (11

In [71]:
df = pd.DataFrame()
for ind, model in enumerate(mod):
    df['mod' + str(ind+1)] = [
        model.rsquared_adj.round(3),
        model.aic.round(3),
        model.bic.round(3)
    ]
df

,mod1,mod2,mod3,mod4,mod5,mod6
0,0.124,0.127,0.124,0.123,0.126,0.125
1,10524.414,10523.475,10525.749,10527.796,10524.675,10527.783
2,10551.771,10555.392,10557.667,10564.273,10561.152,10573.379


In [72]:
df_res = pd.DataFrame(columns=['Метод', 'Регрессия'])
df_res.loc[0] = ['Adj.R2', df.loc[0].idxmax()]
df_res.loc[1] = ['AIC', df.loc[1].idxmin()]
df_res.loc[2] = ['BIC', df.loc[2].idxmin()]
df_res

,Метод,Регрессия
0,Adj.R2,mod2
1,AIC,mod2
2,BIC,mod1
